In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [5]:
from TSFEA import *

In [6]:
tsfea = TSFEA()

# Get Earnings Number to Homogenize Dates across Tickers

In [45]:
homogenized_earnings = tsfea.homogenize_earnings_dates('AllEarningsFmt.csv')
homogenized_earnings.head()

CUSIP       EDATE        LFRI  PrePRC  OpmPRC  PostPRC  \
TICKER ANNDATS                                                               
000R   20191106  14163310  2019-11-06  2019-11-01   11.96   12.02    12.35   
       20190806  14163310  2019-08-06  2019-08-02   10.22   10.71     7.73   
       20190509  14163310  2019-05-09  2019-05-03   16.32   16.40    14.94   
       20190307  14163310  2019-03-07  2019-03-01   24.17   25.54    22.80   
       20181108  14163310  2018-11-08  2018-11-02   18.00   18.10    18.85   

                 ERNUM  
TICKER ANNDATS          
000R   20191106     15  
       20190806     14  
       20190509     13  
       20190307     12  
       20181108     11

# Reconcile Estimates Dates

In [26]:
reconciled_estimates = tsfea.reconcile_estimates_with_earnings(
    'InitialEstimatesData20182022.csv',
    homogenized_earnings
)

In [21]:
reconciled_estimates.head()

OFTIC     CUSIP          CNAME  STATPERS MEASURE FISCALP  FPI  \
TICKER ERNUM                                                                 
00AI   1      WATT  29272C10  ENERGOUS CORP  20210520     EPS     QTR    6   
       1      WATT  29272C10  ENERGOUS CORP  20210617     EPS     QTR    6   
       1      WATT  29272C10  ENERGOUS CORP  20210715     EPS     QTR    6   
       1      WATT  29272C10  ENERGOUS CORP  20210318     EPS     QTR    7   
       1      WATT  29272C10  ENERGOUS CORP  20210415     EPS     QTR    7   

             ESTFLAG  NUMEST  NUMUP  NUMDOWN  MEDEST  MEANEST  STDEV  HIGHEST  \
TICKER ERNUM                                                                    
00AI   1           P       2      1        1   -0.12    -0.12   0.01    -0.11   
       1           P       2      0        0   -0.12    -0.12   0.01    -0.11   
       1           P       2      0        0   -0.12    -0.12   0.01    -0.11   
       1           P       2      1        0   -0.12    -0.12   0.00    -0.12   
       1           P       2      0        0   -0.12    -0.12   0.00    -0.12   

              LOWEST  
TICKER ERNUM          
00AI   1       -0.13  
       1       -0.13  
       1       -0.13  
       1       -0.12  
       1       -0.12

# Reconcile Returns

In [110]:
reconciled_returns = tsfea.reconcile_returns_with_earnings(
    'dsf_save3.csv',
    homogenized_earnings
)

In [111]:
reconciled_returns.head()

CUSIP  PERMNO    PRC   returns
TICKER ERNUM                                   
000R   0      14163310   14378  11.89 -0.037247
       0      14163310   14378  11.96  0.005887
       0      14163310   14378  11.83 -0.010870
       0      14163310   14378  11.87  0.003381
       0      14163310   14378  11.97  0.008425

# Feature Extraction

In [112]:
medest_features = tsfea.extract_features_from_column(
    reconciled_estimates,
    'MEDEST'
)

In [113]:
medest_features.head()

MEDESTDN_HistogramMode_5  MEDESTDN_HistogramMode_10  \
TICKER ERNUM                                                        
00AI   0                  3.980689e-01               9.056067e-01   
       2                           NaN                        NaN   
       3                 -8.881784e-16              -8.881784e-16   
       4                  8.490331e-01               9.515026e-01   
       5                 -5.250000e-01              -6.375000e-01   

              MEDESTCO_f1ecac  MEDESTCO_FirstMin_ac  \
TICKER ERNUM                                          
00AI   0             2.590830                   7.0   
       2             0.000000                   0.0   
       3             0.505696                   2.0   
       4             0.677410                   2.0   
       5             1.704643                   6.0   

              MEDESTCO_HistogramAMI_even_2_5  MEDESTCO_trev_1_num  \
TICKER ERNUM                                                        
00AI   0                            0.547993             0.188513   
       2                                 NaN                  NaN   
       3                            0.693147             0.000000   
       4                            0.500402            -1.423097   
       5                            0.389048             1.265625   

              MEDESTMD_hrv_classic_pnn40  \
TICKER ERNUM                               
00AI   0                        0.214286   
       2                             NaN   
       3                        0.666667   
       4                        0.666667   
       5                        0.333333   

              MEDESTSB_BinaryStats_mean_longstretch1  \
TICKER ERNUM                                           
00AI   0                                        24.0   
       2                                         NaN   
       3                                         2.0   
       4                                         2.0   
       5                                         3.0   

              MEDESTSB_TransitionMatrix_3ac_sumdiagcov  \
TICKER ERNUM                                             
00AI   0                                      0.027778   
       2                                           NaN   
       3                                      0.074074   
       4                                      0.074074   
       5                                      0.166667   

              MEDESTPD_PeriodicityWang_th0_01  ...  \
TICKER ERNUM                                   ...   
00AI   0                                 14.0  ...   
       2                                  0.0  ...   
       3                                  0.0  ...   
       4                                  0.0  ...   
       5                                  2.0  ...   

              MEDESTFC_LocalSimple_mean1_tauresrat  \
TICKER ERNUM                                         
00AI   0                                  0.142857   
       2                                       NaN   
       3                                  1.000000   
       4                                  1.000000   
       5                                  0.250000   

              MEDESTDN_OutlierInclude_p_001_mdrmd  \
TICKER ERNUM                                        
00AI   0                                 0.635294   
       2                                      NaN   
       3                                 0.250000   
       4                                 0.428571   
       5                                 0.700000   

              MEDESTDN_OutlierInclude_n_001_mdrmd  \
TICKER ERNUM                                        
00AI   0                                -0.270588   
       2                                      NaN   
       3                                 0.250000   
       4                                 0.000000   
       5                                -0.300000   

              MEDESTSP_Summaries_welch_rect_area_

In [114]:
return_features = tsfea.extract_features_from_column(
    reconciled_returns,
    'returns'
)

In [115]:
return_features.head()

returnsDN_HistogramMode_5  returnsDN_HistogramMode_10  \
TICKER ERNUM                                                          
000R   0                       0.601841                    0.178407   
       8                       0.624526                    0.076035   
       9                       0.404421                    0.009646   
       10                      0.185965                    0.185965   
       11                     -0.593572                   -0.199666   

              returnsCO_f1ecac  returnsCO_FirstMin_ac  \
TICKER ERNUM                                            
000R   0              0.584964                    1.0   
       8              0.594238                    1.0   
       9              0.681546                    2.0   
       10             0.752280                    3.0   
       11             0.663511                    2.0   

              returnsCO_HistogramAMI_even_2_5  returnsCO_trev_1_num  \
TICKER ERNUM                                                          
000R   0                             0.071439             -0.918074   
       8                             0.101418             -1.002263   
       9                             0.014830              3.319158   
       10                            0.036644             -0.712857   
       11                            0.039674              1.243307   

              returnsMD_hrv_classic_pnn40  \
TICKER ERNUM                                
000R   0                         0.825397   
       8                         0.982759   
       9                         0.937500   
       10                        0.964286   
       11                        0.971831   

              returnsSB_BinaryStats_mean_longstretch1  \
TICKER ERNUM                                            
000R   0                                          4.0   
       8                                          9.0   
       9                                          3.0   
       10                                         6.0   
       11                                         5.0   

              returnsSB_TransitionMatrix_3ac_sumdiagcov  \
TICKER ERNUM                                              
000R   0                                       0.011758   
       8                                       0.001090   
       9                                       0.020576   
       10                                      0.005144   
       11                                      0.019048   

              returnsPD_PeriodicityWang_th0_01  ...  \
TICKER ERNUM                                    ...   
000R   0                                   6.0  ...   
       8                                   8.0  ...   
       9                                   2.0  ...   
       10                                 10.0  ...   
       11                                  2.0  ...   

              returnsFC_LocalSimple_mean1_tauresrat  \
TICKER ERNUM                                          
000R   0                                   1.000000   
       8                                   1.000000   
       9                                   0.200000   
       10                                  0.333333   
       11                                  0.500000   

              returnsDN_OutlierInclude_p_001_mdrmd  \
TICKER ERNUM                                         
000R   0                                 -0.250000   
       8                                  0.050847   
       9                                 -0.224490   
       10                                -0.350877   
       11                                 0.500000   

              returnsDN_OutlierInclude_n_001_mdrmd  \
TICKER ERNUM                                         
000R   0                                 -0.562500   
       8                                  0.220339   
       9                                 -0.163265   
       10                                 0.438596   
       11    

# Prediction Test

### Add Models

In [131]:
tsfea.models = {
    'XGBClf': XGBClassifier(eval_metric='error')
}

tsfea.hyparam_space = {
    'XGBClf': {
        'max_depth': scope.int(hp.quniform("max_depth", 3, 18, 1)),
        'gamma': hp.uniform ('gamma', 0, 9),
        'reg_alpha' : hp.uniform('reg_alpha', 0, 1),
        'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
        'min_child_weight' : scope.int(hp.quniform('min_child_weight', 0, 10, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 100, 1000, 1))
    }
}

for more details on defining hyperparameter spaces, visit http://hyperopt.github.io/hyperopt/getting-started/search_spaces/, or google hyperparameter tuning [ml model] hyperopt.

In [132]:
target_earnings = homogenized_earnings.droplevel(
    1
).set_index(
    "ERNUM",
    append=True
).sort_index()

target_earnings["TARGET"] = pd.to_numeric((target_earnings.PostPRC / target_earnings.PrePRC - 1) >= 0)
target_earnings = pd.DataFrame(
        target_earnings.loc[
        :,
        "TARGET"
    ]
)

In [133]:
df = target_earnings.join(
    return_features, 
    how='inner'
).join(
    medest_features,
    how='inner'
).groupby(
    level=[0, 1]
).fillna(
    method='ffill'
).fillna(
    0
)
df.head()

TARGET  returnsDN_HistogramMode_5  returnsDN_HistogramMode_10  \
TICKER ERNUM                                                                  
00AI   2       False                  -0.853350                   -0.438334   
       3        True                   0.087686                   -0.158264   
       4       False                  -0.092983                    0.123153   
       5       False                  -0.418384                   -0.166993   
       6       False                  -0.060193                   -0.493178   

              returnsCO_f1ecac  returnsCO_FirstMin_ac  \
TICKER ERNUM                                            
00AI   2              0.699683                    2.0   
       3              0.559707                    1.0   
       4              0.524531                    1.0   
       5              0.467555                    1.0   
       6              0.594393                    1.0   

              returnsCO_HistogramAMI_even_2_5  returnsCO_trev_1_num  \
TICKER ERNUM                                                          
00AI   2                             0.024380              2.631426   
       3                             0.098551              0.403508   
       4                             0.117062             -0.210589   
       5                             0.139532              3.508980   
       6                             0.047507             -2.801294   

              returnsMD_hrv_classic_pnn40  \
TICKER ERNUM                                
00AI   2                         0.823529   
       3                         0.980000   
       4                         1.000000   
       5                         0.983871   
       6                         0.974684   

              returnsSB_BinaryStats_mean_longstretch1  \
TICKER ERNUM                                            
00AI   2                                          3.0   
       3                                          6.0   
       4                                          4.0   
       5                                          6.0   
       6                                          8.0   

              returnsSB_TransitionMatrix_3ac_sumdiagcov  ...  \
TICKER ERNUM                                             ...   
00AI   2                                       0.001067  ...   
       3                                       0.006800  ...   
       4                                       0.005711  ...   
       5                                       0.007631  ...   
       6                                       0.001389  ...   

              MEDESTFC_LocalSimple_mean1_tauresrat  \
TICKER ERNUM                                         
00AI   2                                      0.00   
       3                                      1.00   
       4                                      1.00   
       5                                      0.25   
       6                                      0.20   

              MEDESTDN_OutlierInclude_p_001_mdrmd  \
TICKER ERNUM                                        
00AI   2                                 0.000000   
       3                                 0.250000   
       4                                 0.428571   
       5                                 0.700000   
       6                                 0.846154   

              MEDESTDN_OutlierInclude_n_001_mdrmd  \
TICKER ERNUM                                        
00AI   2                                 0.000000   
       3                                 0.250000   
       4                                 0.000000   
       5                                -0.300000   
       6                                -0.384615   

              MEDESTSP_Summaries_welch_rect_area_5_1  \
TICKER ERNUM                                           
00AI   2                                0.000000e+00   
       3                                0.000000e+00   
       4                                8.804251e-

In [134]:
(
    X_train, 
    X_test, 
    y_train, 
    y_test
) = (
    df.loc[(slice(None), slice(1, 13)), df.columns != "TARGET"],
    df.loc[(slice(None), slice(14, 16)), df.columns != "TARGET"],
    df.loc[(slice(None), slice(1, 13)), "TARGET"],
    df.loc[(slice(None), slice(14, 16)), "TARGET"],
)

In [136]:
y_pred = tsfea.tune_train_predict(
    X_train,
    X_test,
    y_train,
    y_test,
    'XGBClf',
    30,
    20
)

100%|██████████| 20/20 [03:05<00:00,  9.27s/trial, best loss: -0.5939850328628498]


In [137]:
metrics.f1_score(y_pred, y_test)

0.540521978021978

# Toy Prediction Test

In [94]:
from sklearn.datasets import load_breast_cancer

In [95]:
data = load_breast_cancer()
X, y = data.data, data.target

In [150]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [151]:
y_pred = tsfea.tune_train_predict(
    X_train,
    X_test,
    y_train,
    y_test,
    'XGBClf',
    15,
    50
)

100%|██████████| 50/50 [00:35<00:00,  1.41trial/s, best loss: -0.9479826194909994]
